In [1]:
!pip install langchain-community langchain-chroma langchain-openai bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)


In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key="")

In [4]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key="...")
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: .... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
retriever = vectorstore.as_retriever()

user_msg = "What is Task Decomposition?"
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

NameError: name 'vectorstore' is not defined

In [ ]:
response = llm.invoke(user_prompt)
print(response.content)

Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This can be achieved using techniques like Chain of Thought (CoT) and Tree of Thoughts, which help in exploring multiple reasoning paths and structuring the thought process. It can be implemented through various prompting methods, task-specific instructions, or human inputs to guide the agent in completing the overarching task effectively.
